<a href="https://colab.research.google.com/github/dadossowhat/bigben/blob/main/diarizacao_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
fonte = "db_bigben_plano-midia"
saida = "db_bigben_diarizacao"
aba_salva = 0

In [ ]:
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
import numpy as np

# Autenticar com o Google
auth.authenticate_user()
creds, _ = default()

# Autorizar acesso ao Google Sheets
gc = gspread.authorize(creds)

# Carregar os dados da planilha
df_plano = pd.DataFrame(gc.open(fonte).get_worksheet(0).get_all_records())

# Converter a coluna "MES/ANO" para datetime
df_plano["MES/ANO"] = pd.to_datetime(df_plano["MES/ANO"], format='%m/%Y')

In [ ]:
# Converter as colunas tipo numérico (adaptar para o nome da coluna desejada)
df_plano["Investimento"] = df_plano["Investimento"].astype(float)
df_plano["Impressoes"] = df_plano["Impressoes"].astype(int)
df_plano["Views"] = df_plano["Views"].astype(int)
df_plano["Cliques"] = df_plano["Cliques"].astype(int)
df_plano["Sessoes"] = df_plano["Sessoes"].astype(int)

In [ ]:
# Obter todas as combinações de "MOTE DA CAMPANHA", "OBJETIVO", "ETAPA" e "VEÍCULO"
combinacoes = df_plano[['MOTE DA CAMPANHA', 'SUBCATEGORIA DE CAMPANHA', 'OBJETIVO', 'ETAPA', 'KPI', 'FORMATO', 'VEÍCULO']].drop_duplicates()

In [ ]:
# Criar DataFrame para expandir os dados
new_data = []

In [ ]:
# Iterar sobre cada combinação
for _, combinacao in combinacoes.iterrows():
    mote_campanha = combinacao['MOTE DA CAMPANHA']
    subcategoria_campanha = combinacao['SUBCATEGORIA DE CAMPANHA']
    objetivo = combinacao['OBJETIVO']
    etapa = combinacao['ETAPA']
    kpi = combinacao['KPI']
    formato = combinacao['FORMATO']
    veiculo = combinacao['VEÍCULO']

    # Filtrar o DataFrame original pela combinação atual
    df_filtrado = df_plano[
        (df_plano['MOTE DA CAMPANHA'] == mote_campanha) &
        (df_plano['SUBCATEGORIA DE CAMPANHA'] == subcategoria_campanha) &
        (df_plano['OBJETIVO'] == objetivo) &
        (df_plano['ETAPA'] == etapa) &
        (df_plano['KPI'] == kpi) &
        (df_plano['FORMATO'] == formato) &
        (df_plano['VEÍCULO'] == veiculo)
    ]

    # Iterar sobre cada linha do DataFrame filtrado
    for index, row in df_filtrado.iterrows():
        # Obter o número de dias no mês atual
        dias_no_mes = row['MES/ANO'].days_in_month

        # Calcular o investimento diário
        investimento_diario = row['Investimento'] / dias_no_mes
        impressoes_diario = row['Impressoes'] / dias_no_mes
        views_bruto_diario = row['Views'] / dias_no_mes
        cliques_diario = row['Cliques'] / dias_no_mes
        sessoes_diario = row['Sessoes'] / dias_no_mes

        # Iterar sobre cada dia do mês
        for day in range(1, dias_no_mes + 1):  # Adicionando +1 para incluir o último dia
            new_row = {
                'DATA': pd.Timestamp(row['MES/ANO'].year, row['MES/ANO'].month, day),
                'MES/ANO': row['MES/ANO'].strftime('%m/%Y'),  # Mantendo o formato MES/ANO
                'MOTE DA CAMPANHA': mote_campanha,
                'SUBCATEGORIA DE CAMPANHA': subcategoria_campanha,
                'OBJETIVO': objetivo,
                'ETAPA': etapa,
                'KPI': kpi,
                'FORMATO': formato,
                'VEÍCULO': veiculo,
                'Investimento': investimento_diario,
                'Impressoes': impressoes_diario,
                'Views': views_bruto_diario,
                'Cliques': cliques_diario,
                'Sessoes': sessoes_diario,
            }
            # Adicionar as métricas diárias calculadas
            new_data.append(new_row)

In [ ]:
new_df = pd.DataFrame(new_data)
# Substituir NaN e infinitos por 0
new_df.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

# Converter a coluna de Timestamp para strings
new_df['DATA'] = new_df['DATA'].dt.strftime('%Y-%m-%d')


In [ ]:
sh = gc.open(saida)

worksheet = sh.get_worksheet(0) #escolher qual aba quer salvar ali coloquei 0 que é primeira aba no caso
worksheet.clear()

worksheet.update([new_df.columns.values.tolist()]+new_df.values.tolist())

{'spreadsheetId': '1XwTX8QRTgie82tvBLHKybTLnQRft7EbM1kFnhsAyW1I',
 'updatedRange': 'diarizacao!A1:N404',
 'updatedRows': 404,
 'updatedColumns': 14,
 'updatedCells': 5656}